In [1]:
import requests

In [32]:
heroes = requests.get("https://api.opendota.com/api/heroes").json()
heroes

[{'id': 1,
  'name': 'npc_dota_hero_antimage',
  'localized_name': 'Anti-Mage',
  'primary_attr': 'agi',
  'attack_type': 'Melee',
  'roles': ['Carry', 'Escape', 'Nuker'],
  'legs': 2},
 {'id': 2,
  'name': 'npc_dota_hero_axe',
  'localized_name': 'Axe',
  'primary_attr': 'str',
  'attack_type': 'Melee',
  'roles': ['Initiator', 'Durable', 'Disabler', 'Carry'],
  'legs': 2},
 {'id': 3,
  'name': 'npc_dota_hero_bane',
  'localized_name': 'Bane',
  'primary_attr': 'all',
  'attack_type': 'Ranged',
  'roles': ['Support', 'Disabler', 'Nuker', 'Durable'],
  'legs': 4},
 {'id': 4,
  'name': 'npc_dota_hero_bloodseeker',
  'localized_name': 'Bloodseeker',
  'primary_attr': 'agi',
  'attack_type': 'Melee',
  'roles': ['Carry', 'Disabler', 'Nuker', 'Initiator'],
  'legs': 2},
 {'id': 5,
  'name': 'npc_dota_hero_crystal_maiden',
  'localized_name': 'Crystal Maiden',
  'primary_attr': 'int',
  'attack_type': 'Ranged',
  'roles': ['Support', 'Disabler', 'Nuker'],
  'legs': 2},
 {'id': 6,
  'name': 

In [38]:
id_to_name = {el["id"]: el["localized_name"] for el in heroes}

In [ ]:
for hero in heroes:
    hero_id = hero["id"]
    hero_stats = requests.get(f"https://api.opendota.com/api/heroes/{hero_id}/matchups").json()
    hero_stats = [{"hero_name": id_to_name[el["hero_id"]], "win_rate": el["wins"] / el["games_played"]} for el in hero_stats]
    hero_stats = sorted(hero_stats, key=lambda x: x["win_rate"], reverse=True)
    best_versus = [el["hero_name"] for el in hero_stats[:10]]
    hero["best_versus"] = best_versus

In [ ]:
import json

with open("../item_ids.json") as f:
    item_ids = json.load(f)


for hero in heroes:
    hero_id = hero["id"]
    hero_name = id_to_name[hero_id]
    items_info = requests.get(f"https://api.opendota.com/api/heroes/{hero_id}/itemPopularity").json()
    best_items = items_info.get("mid_game_items")
    if best_items is None:
        best_items = items_info.get("late_game_items")
    best_items = sorted(best_items.items(), key=lambda x: x[1], reverse=True)
    best_items = [best_items[i][0] for i in range(min(3, len(best_items)))]
    best_items = [item_ids[item_id] for item_id in best_items]
    hero["best_items"] = best_items


In [ ]:
# dota2_relatives = [("Zeus", "Mars"), ("Ember-Spirit", "Storm-Spirit", "Void-Spirit", "Earth-Spirit"), ("Terrorblade", "Nature`s-Profit", "Enchantress"), 
#                    ("Snapfire", "Timbersaw"), ("Winter-Wyvern", "Dragon-Knight", "Jakiro"), ("Bane", "Faceless-Void"), ("Lina", "Crystal-Maiden")]

with open("../dota_relatives.txt") as f:
    dota_relatives = f.readlines()
    dota_relatives = [el.split() for el in dota_relatives]

relatives = dict()

for rel_el in dota_relatives:
    for i in range(len(rel_el)):
        hero_relatives = rel_el[:i]
        if i < len(rel_el) - 1:
            hero_relatives += rel_el[i + 1:]
        relatives[rel_el[i]] = hero_relatives
    
for hero in heroes:
    hero["relatives"] = relatives.get(hero["localized_name"].replace(" ", "-"), [])

In [235]:
relatives

{'Zeus': ['Mars'],
 'Mars': ['Zeus'],
 'Ember-Spirit': ['Storm-Spirit', 'Void-Spirit', 'Earth-Spirit'],
 'Storm-Spirit': ['Ember-Spirit', 'Void-Spirit', 'Earth-Spirit'],
 'Void-Spirit': ['Ember-Spirit', 'Storm-Spirit', 'Earth-Spirit'],
 'Earth-Spirit': ['Ember-Spirit', 'Storm-Spirit', 'Void-Spirit'],
 'Terrorblade': ['Natures-Prophet', 'Enchantress'],
 'Natures-Prophet': ['Terrorblade', 'Enchantress'],
 'Enchantress': ['Terrorblade', 'Natures-Prophet'],
 'Snapfire': ['Timbersaw'],
 'Timbersaw': ['Snapfire'],
 'Winter-Wyvern': ['Dragon-Knight', 'Jakiro'],
 'Dragon-Knight': ['Winter-Wyvern', 'Jakiro'],
 'Jakiro': ['Winter-Wyvern', 'Dragon-Knight'],
 'Bane': ['Faceless-Void'],
 'Faceless-Void': ['Bane'],
 'Lina': ['Crystal-Maiden'],
 'Crystal-Maiden': ['Lina']}

In [236]:
cleaned_heroes = []

for hero in heroes:
    new_hero = {"name": hero["localized_name"].replace(" ", "-"), "attack_type": hero["attack_type"], "roles": hero["roles"]}
    attr = hero["primary_attr"]
    subclass = {"agi": "agile_hero", "str": "strong_hero", "all": "universal_hero", "int": "intellectual_hero"}[attr]
    new_hero["subclass"] = subclass
    new_hero["items"] = hero["best_items"]
    cleaned_best_bersus = [el.replace(" ", "-") for el in hero["best_versus"]]
    new_hero["best_versus"] = cleaned_best_bersus
    new_hero["relatives"] = hero["relatives"]

    cleaned_heroes.append(new_hero)

cleaned_heroes

[{'name': 'Anti-Mage',
  'attack_type': 'Melee',
  'roles': ['Carry', 'Escape', 'Nuker'],
  'subclass': 'agile_hero',
  'items': ['blade_of_alacrity', 'yasha', 'diadem'],
  'best_versus': ['Phantom-Lancer',
   'Enchantress',
   'Naga-Siren',
   'Broodmother',
   'Arc-Warden',
   'Chaos-Knight',
   'Faceless-Void',
   'Razor',
   'Death-Prophet',
   'Visage'],
  'relatives': []},
 {'name': 'Axe',
  'attack_type': 'Melee',
  'roles': ['Initiator', 'Durable', 'Disabler', 'Carry'],
  'subclass': 'strong_hero',
  'items': ['blink', 'ogre_axe', 'blade_mail'],
  'best_versus': ['Lone-Druid',
   'Viper',
   'Bristleback',
   'Riki',
   'Anti-Mage',
   'Naga-Siren',
   'Wraith-King',
   'Silencer',
   'Tinker',
   'Skywrath-Mage'],
  'relatives': []},
 {'name': 'Bane',
  'attack_type': 'Ranged',
  'roles': ['Support', 'Disabler', 'Nuker', 'Durable'],
  'subclass': 'universal_hero',
  'items': ['arcane_boots', 'staff_of_wizardry', 'aether_lens'],
  'best_versus': ['Necrophos',
   'Warlock',
   '

In [ ]:
from rdflib import Graph, URIRef, Literal, RDF, Namespace

# Создаем граф и загружаем существующую онтологию
g = Graph()
g.parse("../empty_ontology.rdf", format="xml")  # Замените путь на путь к вашему RDF файлу

# Определяем пространство имен
NS = Namespace("http://www.semanticweb.org/root/ontologies/2025/0/dota2#")
g.bind("dota2", NS)

In [240]:
# Добавляем данные в граф (существующую онтологию)
for hero_data in cleaned_heroes:
    hero_uri = URIRef(NS[hero_data["name"]])  # Создаем уникальный идентификатор для героя
    
    # Устанавливаем основной класс hero
    # g.add((hero_uri, RDF.type, URIRef(NS.hero)))
    g.add((hero_uri, NS.hero_name, Literal(hero_data["name"])))
    g.add((hero_uri, NS.hero_attack_type, Literal(hero_data["attack_type"])))
    
    # Устанавливаем подкатегорию (subclass) для героя
    subclass_uri = URIRef(NS[hero_data["subclass"]])  # Получаем URI подкласса
    g.add((hero_uri, RDF.type, subclass_uri))  # Добавляем индивид как тип подкласса (не создаем новый класс)

    # Связи с ролями (если роли существуют в онтологии)
    for role_name in hero_data["roles"]:
        role_uri = URIRef(NS[role_name])
        # Убедимся, что роль существует, если нет — добавим ее
        g.add((role_uri, RDF.type, URIRef(NS.role)))  # Убедимся, что роль - это экземпляр класса role
        g.add((hero_uri, URIRef(NS.hasRole), role_uri))
        g.add((role_uri, NS.role_name, Literal(role_name)))
    
    # Связи с расами (если расы существуют в онтологии)
    for item_name in hero_data["items"]:
        item_uri = URIRef(NS[item_name])
        g.add((item_uri, RDF.type, URIRef(NS.item)))
        g.add((hero_uri, URIRef(NS.buysItem), item_uri))
        g.add((item_uri, NS.item_name, Literal(item_name)))
    
    # Отношение bestVersus
    for opponent in hero_data["best_versus"]:
        opponent_uri = URIRef(NS[opponent])
        g.add((hero_uri, URIRef(NS.bestVersus), opponent_uri))
    
    for relative in hero_data["relatives"]:
        relative_uri = URIRef(NS[relative])
        g.add((hero_uri, URIRef(NS.relativeTo), relative_uri))
        

# Сохраняем изменения обратно в файл (перезаписываем)
g.serialize("dota2_ontology_v2.rdf", format="xml")

<Graph identifier=N02b8af0228964cd2a79da7d1d17295f3 (<class 'rdflib.graph.Graph'>)>